In [ ]:
!pip install plotly==4.5.2

# Correlation

We often want to see if there is a relationship between two or more variables. As one variable increases, does the other regularly increase, decrease, or not behave in any particular way. 

If, as the first variable increases, the second variable behaves in a normal way - regularly increasing or decreasing - we say that there is a correlation. We call the first variable the independent variable and usually plot it on our x-axis. We call the second variable the dependent variable, and usually plot it on our y-axis. 

The classic example of an independent variable is time. As time moves forward (increases), does the amount of CO2 in the atmosphere go up or down? As time moves forward, does our bank account get bigger or smaller? Although time is the classic example, the independent variable can be almost anything. As CO2 levels increase, does the temperature of the earth go up or down? As a country's GDP increases, do the lifespan of it's citizens go up or down?

The item that we want to measure the behavior of is the dependent variable - whether it's money, temperature, CO2, or anything else.

## Measuring Correlation
We not only want to see if there is a correlation, we want to measure it. The most common measure of correlation is called the correlation coefficient and is usually denoted by a single lowercase *r*. The correlation coefficient goes from -1 to 0 to +1. 
- a correlation coefficient coefficient of -1 means there is a perfect negative linear correlation between the variables.
- a correlation coefficient of 0 means there is no correlation between the variables.
- a correlation coefficient of +1 means there is a perfect linear positive correlation between the variables

### Examples
 I'm going to display some graphs that illustrate correlation. I'm just going to use made up data so the point is clear. Don't worry about the code here too much, just pay attention to the graphs.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
X=np.linspace(0,20,40)
px.scatter(x=X,y=X,title='Perfect Positive Correlation (r=1)')

In [ ]:
y=X+np.random.normal(size=40)
px.scatter(x=X,y=y,title=f'Strong Positive Correlation (r={np.corrcoef(X,y)[0,1]})')

In [ ]:
y=X+np.random.normal(scale=10,size=40)
px.scatter(x=X,y=y,title=f'Some Positive Correlation (r={np.corrcoef(X,y)[0,1]})')

In [ ]:
y=np.random.normal(scale=10,size=40)
px.scatter(x=X,y=y,title=f'(Almost) No Correlation (r={np.corrcoef(X,y)[0,1]})')

In [ ]:
y=-X+np.random.normal(scale=10,size=40)
px.scatter(x=X,y=y,title=f'Some Negative Correlation (r={np.corrcoef(X,y)[0,1]})')

In [ ]:
y=-X+np.random.normal(size=40)
px.scatter(x=X,y=y,title=f'Strong Negative Correlation (r={np.corrcoef(X,y)[0,1]})')

In [ ]:
y=-X
px.scatter(x=X,y=y,title=f'Perfect Negative Correlation (r={np.corrcoef(X,y)[0,1]})')

# Linear models

If there is a correlation between our variables, we often want to create a model to explain the correlation. This allows us to predict the value of our dependent variable for values of our independent variable that we haven't seen yet. What will the temperature be in 2030? If a country's GDP increases by $10 billion, how long will their people live for?

The simplest model is the linear model. Even though it is simple, it is still very useful. A linear model is just a straight line we make using our data points. We choose a line that minimizes the distance to all of the points. If we were to do this by hand it would take quite a bit of work, so, of course, we use our python tools. In this case, we will use a package called statsmodels that is used for data analysis.

Once we have our linear model, we need to know how to interpret it. Remember that we normally define a line with the equation:

$$ \begin{align} y &= mx + b\\
&\textrm{where}\\
y &= \textrm{our dependent variable} \\
x &= \textrm{our independent variable}\\
m &= \textrm{the } slope\textrm{, or the rate of change}\\
b &= \textrm{the } y-intercept\textrm{, or the starting value}\\
\end{align}
$$

The key things here are *m*, which represents the rate of change of our dependent variable, and *b* which represents our starting value of our dependent variable.

## Boston Housing Example
First, we need to import statsmodels:

In [ ]:
import statsmodels.api as sm

In this example we'll be looking at data about the price of houses in Boston. This is a classic (old) dataset from 1978, so the prices don't make a lot of sense. It will still be useful to look at. We need to import our data and look at it.

In [ ]:
boston=pd.read_csv('https://raw.githubusercontent.com/SkyIslandsMath/semester-2/master/data/boston_mod.csv')
boston

We can see we have a lot of columns of data here. Each row represents a different area of the city. Let's focus on two of the columns here, `'MEDV'` and `'RM'`, which indicate the median value of homes and average number of rooms in an area. I would think that, in general, as the number of rooms in a house goes up, the value of the house would also go up. Let's see if this is true, and if we can quantify it. 

In order to make our code easier to read, I'm going to get rid of our other columns. This works a lot like making a mask for our data:

In [ ]:
# I only want to look at the 'RM' and 'MEDV' columns, so I'll put them in a list
cols=['RM','MEDV']
#now I use the list on my dataframe just like a mask
boston=boston[cols]
boston

Our First job is to see if there is a correlation and to measure what it is.

In this case, the number of rooms is our independent variable (x-axis) and the value of the house is our dependent variable (y-axis).

In [ ]:
px.scatter(boston,
           x='RM',
           y='MEDV',
           title='Boston Home Values in 1978',
           labels={
               'RM': 'Average number of rooms',
               'MEDV': 'Median home value ($1,000s)'
           })

We can see that there does appear to be a positive correlation between the number of houses and the value of homes in an area. In order to quantify it we can use pandas correlation coefficient function `.corr()`:

In [ ]:
boston[cols].corr()

This finds the correlation between each combination of columns. When we compare `'RM'` to `'RM'` we get a correlation coefficient of 1 because they are the exact same values. The same happens when we compare `'MEDV'` to `'MEDV'`. The coefficient we're interested in is the one we get when we compare `'RM'` to `'MEDV'`. We can see that it is `0.686634` which is a moderate correlation.

### Making a linear model
We've already imported statsmodels as `sm`, now we'll use the `OLS` function to make our model. `OLS` stands for ordinary least squares, which is the standard method for making a linear model. 

Before we make our model, we need to isolate our dependent variable (which we will call `X`) and our independent variable (which we will call `Y`). 

Because of the way statsmodels works, we also need to 'add a constant' to our X values, otherwise our y-intercept will always be zero:

In [ ]:
X=boston['RM']
Y=boston['MEDV']

#adding a constant to X so our y-intercept can be non-zero

X=sm.add_constant(X)
X

Now we need to:
1. Make our model
2. fit the model

In [ ]:
# to create our model we just call sm.OLS() with our dependent and independent variables:
model=sm.OLS(Y,X)

#to fit our model, we call the .fit() method with our X and Y variables
model=model.fit()

We can see the results of fitting the model by using `.summary()` on our `model`:

In [ ]:
model.summary()

That's a lot of information that you don't need to understand (I don't understand most of it). The important information to notice right now are the values of in the **coef** column in the middle of the table.  
- We can see from that column that the coefficient of our constant (**`const`**) is `-30.0051`. This means that our y-intercept is at -30. Another interpretation is that a house with zero rooms is worth -\$30,000. 
- We can see that the coefficient of `'RM'` is 8.2686. This means that the slope of our linear model is 8.2686. So, each room adds \$8,268.60 on to the value of the house.

#### Prediction
The largest average house in our data had 8.78 rooms. What would we predict the value of a house to be if it had 11 rooms? In order to do this, we could make a function based on our new linear model: $y = 8.2686x -30.0051$ , or we could use statsmodels built-in predict function. Let's do it both ways:

In [ ]:
rooms=11
(8.2686*rooms)-30.0051

In [ ]:
#because of the quirks of statsmodels, we have to pass a list where the first value is a one and the second is the 
#number of rooms
rooms=11
model.predict([1,rooms])

### Visualizing the model

In order to visualize the model we need to save the scatterplot of rooms versus home values to a variable I'll call `fig`. We'll also need to import a new plotly module: `plotly.graph_objects as go`. This will allow us to add our linear model on top of it later, but our scatterplot will not display untill we tell it to. This is fine.

In [ ]:
import plotly.graph_objects as go
#make our predictions:
y=model.predict(X)
#creating the same scatterplot as above
fig=px.scatter(boston,
           x='RM',
           y='MEDV',
           title='Boston Home Values in 1978',
           labels={
               'RM': 'Average number of rooms',
               'MEDV': 'Median home value ($1,000s)'
           })

# add the linear model on top using go.Scatter():
fig.add_trace(go.Scatter(x=boston['RM'],y=y,name='model'))

## Assignment
Look at the dataset found at 
```
'https://raw.githubusercontent.com/SkyIslandsMath/semester-2/master/data/sea-level.csv'
```
This shows the change in ocean sea level relative to 1880. 

1. Load and look at your data.
2. Plot the data with the year on the x-axis and the sea level on the y-axis.
3. Is there a correlation? What is the correlation coefficient? What does this coefficient mean?
4. Use statsmodels to make a linear model of the data.
5. Print out the summary of the model. What do the coefficients mean?
6. Graph the data points and the model of the data. How does it look?
7. According to this model, what will the sea level be in 2030?